In [ ]:
!pip install -q pyiceberg

In [ ]:
from pyiceberg.catalog.rest import RestCatalog
import logging

import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from device_code_auth import authenticate_device_flow_with_pkce

CATALOG_URL = "http://lakekeeper:8181/catalog"
WAREHOUSE = "test-warehouse"

In [ ]:
# Login as user "Peter" to try reading 'product' namespace
auth_response = authenticate_device_flow_with_pkce(
    token_endpoint="http://keycloak:8080/realms/iceberg/protocol/openid-connect/token",
    device_endpoint="http://keycloak:8080/realms/iceberg/protocol/openid-connect/auth/device",
    client_id="lakekeeper",
    scope="lakekeeper",
)

In [ ]:
access_token = auth_response["access_token"]

In [ ]:
catalog = RestCatalog(
    name="lakekeeper",
    warehouse=WAREHOUSE,
    uri="http://lakekeeper:8181/catalog/",
    token=access_token,
)

In [ ]:
# List namespaces Peter can access
catalog.list_namespaces()

In [ ]:
# List tables Peter can access -> only 1 table granted
list(".".join(t) for t in catalog.list_tables(namespace=("product",)))

In [ ]:
# Read table that Peter has access to
table = catalog.load_table("product.raw_product")
data = table.scan()

In [ ]:
df = data.to_pandas()
df.head()

In [ ]:
# Try insert some data to the table
# Making sure schema matches -> id is INT (int32) and not LONG (int64)
# Must throw PERMISSION_DENIED
table.append(pa.Table.from_pandas(
    pd.DataFrame({"id": [3], "description": ["I can't"], "price": [0.0]}).astype({"id": "int32"})
))

In [ ]:
# Try read table that Peter DOES NOT have access to
# Must throw PERMISSION_DENIED
catalog.load_table("product.enriched_product")